In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar as cal
from collections import OrderedDict

%matplotlib inline
sns.set()
pd.set_option('display.float_format', lambda x: '%.2f' % x)

###Cleaning the Data

- We create labels and ranges for age groups
- Remove any data points with ages less than 0 (not possible) and drop all duplicates that has the patient appearing mulitple times on the same day (can be an error or actual scheduling on the same day, remove all just to be sure).


In [ ]:
bins = [0, 1, 4, 9, 13, 18, 35, 55, np.inf]
labels = ['baby', 'toddler', 'gradeschool', 'preteen', 'teenager', 'youngadult', 'middle-aged', 'elder']
patient_headers = ['PatientId']
cats = ['Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']

data = (pd.read_csv('noshowappointments-kagglev2-may-2016.csv', encoding='utf8', dtype={'PatientId': object, 'AppointmentID': object})
            .query("Age >= 0")
            .drop_duplicates(subset=['PatientId', 'AppointmentDay'], keep=False)
            .assign(ScheduledDay=lambda x: pd.to_datetime(pd.to_datetime(x['ScheduledDay']).dt.date),
                    AppointmentDay=lambda x: pd.to_datetime(x['AppointmentDay']),
                    age_group=lambda x: pd.cut(x['Age'], bins, labels=labels, include_lowest=True),
                    yes=lambda x: (x['No-show'] == 'Yes').astype(int),
                    no=lambda x: (x['No-show'] == 'No').astype(int)))

def noshow_counts(df, columns):
    df = df.copy()
    return df.groupby(columns)['No-show'].value_counts().unstack(1).reset_index()

def get_noshow_ratio(df):
    return df['Yes']/(df['No']+df['Yes'])

def get_percentage(data, columns):
    rows = []
    for column in columns:
        filtered = data[column] != 1
        row = OrderedDict()
        row['Category'] = column
        row['No'] = data[filtered]['no'].sum()
        row['Yes'] = data[filtered]['yes'].sum()
        row['Total'] = row['Yes'] + row['No']
        row['Perc'] = row['No'] / float(row['Total'])
        rows.append(row)
    return pd.DataFrame(rows, columns=rows[0].keys())

def get_levels(df, column):
    levels = df[column].unique()
    rows = []
    for i in levels:
        row = OrderedDict()
        row['level'] = i
        row['no'] = df[df[column] == i]['no'].sum()
        row['yes'] = df[df[column] == i]['yes'].sum()
        row['total'] = float(row['no']+row['yes'])
        row['no_pct'] = row['no']/row['total']
        row['yes_pct'] = row['yes']/row['total']
        rows.append(row)
        
    new_df = pd.DataFrame(rows, columns=row.keys())
    
    if column in df.select_dtypes(include=['category']):
        new_df['level'] = new_df['level'].astype('category')
        new_df['level'].cat.reorder_categories(df[column].cat.categories, inplace=True)

    sns.barplot(x='level', y='no_pct', data=new_df)
    sns.plt.title('Percentage of {} by Level'.format(column))
    sns.plt.ylabel('Percentage')

#no_cats_matched = data[data[cats].apply(lambda x: min(x) == max(x), 1)]
data['no_cat'] = data[cats].apply(lambda x: min(x) == max(x), 1)
data['day'] = data['AppointmentDay'].apply(lambda x: cal.day_name[x.weekday()]).astype('category')
data['day'].cat.reorder_categories(list(cal.day_name)[:6], inplace=True)
data.age_group.cat.reorder_categories(labels, ordered=True, inplace=True)
cats.append('no_cat')

## Changes to the Data Read

- Change PatientId and AppointId to object types (they were coming in as integers and made queryies awkward)
- Make a query for ages greater than or equal to 0 (some entries had ages of -1, and I wasn't sure what that exactly spoke to)
- Drop duplicates of PatientId and AppointmentDay. (Mutiple entires for the patient on the same day)
- Create/Changed several columns:
    - ScheduledDay => Changed type to date_time
    - age_group => Added an age group category that categorizes segments of ages
    - Yes => An explicit yes on attendance
    - No => An explicit no on attendance

In [ ]:
data['day'].cat.reorder_categories(list(cal.day_name)[:6], inplace=True)
#list(cal.day_name)[:6]

In [ ]:
for cat in cats:
    print("{} => {}".format(cat, data[cat].unique()))

## Breakdown by Category

I performed a simple calculation to see what are the percentage of people who came in under one of the categories did not come in. Patients who exhibited Hyptertension came in the most times for their appointments, while peopel who received SMS's did not. However, the percentages are still close, and don't seem to show any patterns that can show the likelihood of a patient appearing.

Some other things to look at regarding categories:
- Percentage of patients that came in who didn't fit any categories
- Percentage of patients who came in with multiple categories

In [ ]:
cats_percentage = data.pipe(get_percentage, cats)
sns.barplot(x='Category', y='Perc', data=cats_percentage)

In [ ]:
def get_levels(df, column):
    levels = df[column].unique()
    rows = []
    for i in levels:
        row = OrderedDict()
        row['level'] = i
        row['no'] = df[df[column] == i]['no'].sum()
        row['yes'] = df[df[column] == i]['yes'].sum()
        row['total'] = float(row['no']+row['yes'])
        row['no_pct'] = row['no']/row['total']
        row['yes_pct'] = row['yes']/row['total']
        rows.append(row)
        
    new_df = pd.DataFrame(rows, columns=row.keys())
    
    if column in df.select_dtypes(include=['category']):
        new_df['level'] = new_df['level'].astype('category')
        new_df['level'].cat.reorder_categories(df[column].cat.categories, inplace=True)

    sns.barplot(x='level', y='no_pct', data=new_df)
    sns.plt.title('Percentage of {} by Level'.format(column))
    sns.plt.ylabel('Percentage')

get_levels(data, 'day')

In [ ]:
data['day'].dtype == 'category'

In [ ]:
get_levels(data, 'Scholarship')

In [ ]:
get_levels(data, 'Hipertension')

In [ ]:
get_levels(data, 'age_group')

In [ ]:
get_levels(data, 'day')

In [ ]:
get_levels(data, 'Age')

In [ ]:
data['day'] = data['AppointmentDay'].apply(lambda x: cal.day_name[x.weekday()])
data

In [ ]:
data['age_group'].value_counts().plot.bar()

In [ ]:
age_groups = data.groupby('age_group')
age_groups['Alcoholism'].sum()

In [ ]:
data[['PatientId', 'AppointmentDay']].duplicated()

## Things to do

1. Column DataType Changes
    - PatientId = Object (or a ID type if pandas has that)
    - Scholarship = Category
    - Hipertension = Category
    - Diabetes = Category
    - Alcoholism = Category
    - Handicap = Category
    - SMS_received = Category
    - No_Shows = Category
    
2. Remove outliers
    - a -1.00 age needs to be removed. 0 age may refer to babies, so that may stay the same.
    
3. Visualizations
    - Bar chart of major categories (Scholarship, Hipertension, Diabetes, Alcoholism, and Handcap)

In [ ]:
data.info()

In [ ]:
major_cats_alt = ['Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received']
major_cats_alt_no_show = ['Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show']
no_shows = data[data['No-show'] == 'Yes']
no_shows.mean(numeric_only=True)

In [ ]:
shows = data[data['No-show'] == 'No']
shows.std(numeric_only=True)

In [ ]:
age_comparison = data[['Age', 'No-show']]
age_comparison.info()

In [ ]:
bins = [0, 1, 4, 9, 13, 18, 35, 55, np.inf]
labels = ['baby', 'toddler', 'gradeschool', 'preteen', 'teenager', 'youngadult', 'middle-aged', 'elder']
data['age_groups'] = pd.cut(data['Age'], bins, labels=labels, include_lowest=True)

age_comparison = (data.pipe(noshow_counts, ['age_groups'])
                      .set_index('age_groups')
                      .fillna(0)
                      .assign(noshow_ratio=get_noshow_ratio))

age_comparison.plot.bar()
age_comparison

In [ ]:
data[major_cats_alt_no_show].groupby('No-show').sum().T.plot.bar()

In [ ]:
data[major_cats_alt_no_show].groupby('No-show').sum()

In [ ]:
data[major_cats_alt].filter?

In [ ]:
data[major_cats_alt].filter

In [ ]:
no_select = data.copy()
no_select['No-selection'] = data.filter(major_cats_alt).eq(0).all(1)
no_select

In [ ]:
selections = data.copy()
selections['Selections'] = data[major_cats_alt].sum(axis=1)
selections = selections.groupby('Selections')['No-show'].value_counts().unstack(1)
selections.apply(lambda x: x/selections.sum().sum())

In [ ]:
data['PatientId'].value_counts().max()

In [ ]:
patient_counts = data.copy()
patient_counts['freq'] = data.groupby('PatientId')['PatientId'].transform('count')
patient_counts = patient_counts.groupby('freq')['No-show'].value_counts().unstack(1).reset_index().fillna(0)
patient_counts['Patients'] = patient_counts.apply(lambda x: (x['No']+x['Yes'])/x['freq'], axis=1)
patient_counts['% of Missed'] = patient_counts['Yes']/(patient_counts['No']+patient_counts['Yes'])
patient_counts.plot.scatter(x='freq', y='% of Missed', marker='o', c='b')

In [ ]:
sns.lmplot(x='freq', y='% of Missed', data=patient_counts)

In [ ]:
gender = data.groupby('Gender')['No-show'].value_counts().unstack(1)
gender

In [ ]:
neighbourhoods = data.groupby('Neighbourhood')['No-show'].value_counts().unstack(1)
neighbourhoods['% of Missed'] = neighbourhoods['Yes']/(neighbourhoods['No']+neighbourhoods['Yes'])
neighbourhoods['total'] = neighbourhoods.iloc[:, :2].sum(axis=1)
neighbourhoods.sort_values('% of Missed', ascending=False)

In [ ]:
data.info()

In [ ]:
test = data.copy()
test['ScheduledDay'] = pd.to_datetime(pd.to_datetime(test['ScheduledDay']).dt.date)
test['AppointmentDay'] = pd.to_datetime(test['AppointmentDay'])
test['days_left'] = test['AppointmentDay'] - test['ScheduledDay']
test = test.groupby('days_left')['No-show'].value_counts().unstack(1).fillna(0).reset_index()
test = test.query('days_left >= "0 days"')
q = pd.cut(test.days_left, 15)
new = test.groupby(q).sum()
new['%'] = new['Yes']/(new['No']+new['Yes'])
new

In [ ]:
data.groupby('No-show')['No-show'].transform('value_counts')